# Extract a CATIA V5 Part

**The old way:** Open CATIA on a Windows workstation, navigate the feature tree, manually read parameters, export views one by one.

**The new way:** Upload a `.CATPart` to Istari, run extraction, get parameters, mass properties, and rendered views automatically.

This notebook:
1. Navigates the Istari system hierarchy
2. Runs CATIA V5 data extraction on a `.CATPart` file
3. Views the extracted parameters, mass properties, and rendered views
4. Snapshots the results so the team can see what was extracted

See [`example-output/`](example-output/) for pre-computed results.

In [ ]:
# Setup
import sys, json
from pathlib import Path

try:
    import istari_digital_client
except ImportError:
    !pip install istari-digital-client python-dotenv -q

repo_root = str(Path.cwd().parent.parent)
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)

from istari_client import get_client

client = get_client()
user = client.get_current_user()
print(f"Connected as: {user.display_name} ({user.email})")

In [ ]:
# Explore the system — navigate the Istari hierarchy
#
# This system tracks one CATIA V5 part file.
#
# Istari links:
#   System: https://demo.istari.app/systems/55726539-5956-447b-8d46-da67aca7ccbe

SYSTEM_ID = "55726539-5956-447b-8d46-da67aca7ccbe"  # Example: Extract CATIA V5 Part
CONFIG_ID = "8f11a945-4a05-4a60-88ab-f0d44b5501f7"  # Baseline configuration

system = client.get_system(SYSTEM_ID)
print(f"System: {system.name}")
print(f"  {system.description}\n")

configs = client.list_system_configurations(SYSTEM_ID, page=1, size=50)
print(f"Configurations ({configs.total}):\n")

for config in configs.items:
    print(f"  {config.name}")
    print(f"    Config ID: {config.id}")

    tracked = client.list_tracked_files(config.id, page=1, size=50)
    print(f"    Tracked files ({tracked.total}):")
    for tf in tracked.items:
        mode = tf.specifier_type.value
        print(f"      {tf.file_id} ({mode})")

    snapshots = client.list_snapshots(configuration_id=config.id, page=1, size=10)
    print(f"    Snapshots ({snapshots.total}):")
    for snap in snapshots.items:
        tags = client.list_tags(snapshot_id=snap.id, page=1, size=10)
        tag_names = [t.tag for t in tags.items]
        tag_str = f"  [{', '.join(tag_names)}]" if tag_names else ""
        revs = client.list_snapshot_revisions(snap.id, page=1, size=50)
        print(f"      {snap.id[:8]}...{tag_str}  ({revs.total} file(s))")
        for r in revs.items:
            size_kb = r.size / 1024
            print(f"        - {r.name} ({size_kb:.1f} KB)")

In [ ]:
# Load the model — show file details and revision history

MODEL_ID = "0e035383-3a51-48fa-8682-29213dda65ab"  # Bracket.CATPart

model = client.get_model(MODEL_ID)
name = model.display_name or model.file.revisions[0].name
print(f"Model: {name}")
print(f"  File ID:   {model.file.id}")
print(f"  Revisions: {len(model.file.revisions)}")
for rev in model.file.revisions:
    size_kb = rev.size / 1024
    ver = f"  ({rev.version_name})" if hasattr(rev, 'version_name') and rev.version_name else ""
    print(f"    {rev.name} — {size_kb:.1f} KB{ver}")
print(f"  Artifacts: {len(model.artifacts)}")
print(f"\nTracked in config: {CONFIG_ID[:8]}... (LATEST)")

In [ ]:
# Run CATIA V5 extraction
#
# This sends the .CATPart file to an Istari agent running CATIA V5.
# The agent extracts: parameters, mass properties, BOM, rendered views, and OBJ mesh.
#
# Docs: https://docs.istaridigital.com/integrations/CAD/dassault_catia_v5

from time import sleep
from datetime import datetime
from istari_digital_client import JobStatusName

print("Submitting CATIA V5 extraction job...")
job = client.add_job(
    model_id=MODEL_ID,
    function="@istari:extract",
    tool_name="dassault_catia_v5",
    tool_version="6R2023",
    operating_system="Windows Server 2022",
    parameters={},
)
print(f"Job: {job.id}")

while True:
    sleep(10)
    job = client.get_job(job.id)
    ts = datetime.now().strftime("%H:%M:%S")
    status = job.status.name.value
    print(f"  [{ts}] {status}")
    if job.status.name in {JobStatusName.COMPLETED, JobStatusName.FAILED}:
        break

if job.status.name == JobStatusName.COMPLETED:
    print("\nExtraction complete!")
else:
    print("\nExtraction failed!")

In [ ]:
# View extraction results
#
# The agent extracted: parameters, mass properties, BOM, 7 PNG views, OBJ mesh.

from IPython.display import HTML, display

model = client.get_model(MODEL_ID)
print(f"Artifacts: {len(model.artifacts)}\n")

for a in model.artifacts:
    rev = a.file.revisions[0] if a.file.revisions else None
    if rev:
        size_kb = rev.size / 1024
        ext = rev.name.split('.')[-1].lower() if '.' in rev.name else ''
        print(f"  {rev.name} ({size_kb:.1f} KB)")
        
        # Show JSON contents inline
        if ext == 'json':
            try:
                data = json.loads(a.read_text())
                print(f"    {json.dumps(data, indent=2)[:500]}")
                if len(json.dumps(data, indent=2)) > 500:
                    print("    ...")
            except Exception:
                pass
            print()

In [ ]:
# Snapshot: capture state after extraction

from istari_digital_client import NewSnapshot, NewSnapshotTag

print("Creating snapshot: post-extraction...")
snap_response = client.create_snapshot(CONFIG_ID, NewSnapshot())
snapshot = snap_response.actual_instance

if hasattr(snapshot, "id"):
    snap_id = snapshot.id
    print(f"  New snapshot: {snap_id[:8]}...")
else:
    snaps = client.list_snapshots(configuration_id=CONFIG_ID, page=1, size=1)
    snap_id = snaps.items[0].id
    print(f"  No changes — tagging existing snapshot: {snap_id[:8]}...")

client.create_tag(snap_id, NewSnapshotTag(tag="post-extraction"))
print("  Tagged: post-extraction")

revs = client.list_snapshot_revisions(snap_id, page=1, size=50)
print(f"  Files: {revs.total}")

In [ ]:
# View system state — all snapshots with tags, files, and sizes

print(f"System: {system.name}")
print(f"  {system.description}\n")

configs = client.list_system_configurations(SYSTEM_ID, page=1, size=50)
for config in configs.items:
    print(f"Configuration: {config.name}")
    print(f"  ID: {config.id}")

    tracked = client.list_tracked_files(config.id, page=1, size=50)
    print(f"  Tracked files: {tracked.total}")

    snapshots = client.list_snapshots(configuration_id=config.id, page=1, size=20)
    print(f"\n  Snapshots ({snapshots.total}):\n")

    for i, snap in enumerate(snapshots.items):
        tags = client.list_tags(snapshot_id=snap.id, page=1, size=10)
        tag_names = [t.tag for t in tags.items]
        tag_str = ", ".join(tag_names) if tag_names else "untagged"
        revs = client.list_snapshot_revisions(snap.id, page=1, size=50)

        print(f"    {i+1}. [{tag_str}]  ({revs.total} files)")
        for r in revs.items:
            size_kb = r.size / 1024
            if size_kb > 1024:
                print(f"       {r.name} ({size_kb/1024:.1f} MB)")
            else:
                print(f"       {r.name} ({size_kb:.1f} KB)")
        print()

## Summary

**What we did:**
1. Uploaded a CATIA V5 `.CATPart` file to Istari
2. Ran automated extraction — parameters, mass properties, BOM, rendered views
3. Viewed the results without ever opening CATIA
4. Snapshotted the extraction results for the team

### Version History

| # | Snapshot Tag | Files | What happened |
|---|-------------|-------|---------------|
| 1 | `initial-upload` / `baseline` | 1 | Raw .CATPart uploaded |
| 2 | `post-extraction` | ~10 | CATPart + extracted parameters, mass, views, OBJ |

### What Istari Did

| Step | Inner Loop (CATIA V5) | Outer Loop (Istari) |
|------|----------------------|---------------------|
| Upload part | — | Stored .CATPart, tracked as LATEST |
| Extract data | Opened part in CATIA V5, read parameters, calculated mass, rendered views | Ran job on Windows agent, stored all artifacts |
| Snapshot | — | Captured point-in-time state with all artifacts |

### What’s Next?

- **Update parameters** — use `@istari:update_parameters` to modify CAD parameters without opening CATIA
- **Extract assemblies** — upload a `.CATProduct` (as .zip) to extract BOM and assembly structure
- **Compare revisions** — upload a modified part, re-extract, and compare parameters across versions